In [ ]:
from spf.model_training_and_inference.models.single_point_networks_inference import (
    convert_datasets_config_to_inference,
    load_model_and_config_from_config_fn_and_checkpoint,
)
import torch
from spf.scripts.train_single_point import load_dataloaders

from tqdm import tqdm

config_fn = "/home/mouse9911/gits/spf/nov2_checkpoints/nov2_small_paired_checkpoints_inputdo0p3/config.yml"
checkpoint_fn = "/home/mouse9911/gits/spf/nov2_checkpoints/nov2_small_paired_checkpoints_inputdo0p3/best.pth"

config_fn = (
    "/home/mouse9911/gits/spf/nov2_checkpoints/paired_checkpoints_inputdo0p3/config.yml"
)
checkpoint_fn = (
    "/home/mouse9911/gits/spf/nov2_checkpoints/paired_checkpoints_inputdo0p3/best.pth"
)
ds_fn = "/mnt/4tb_ssd/nosig_data/wallarrayv3_2024_08_21_10_30_58_nRX2_bounce_spacing0p05075.zarr"

# load model
model, config = load_model_and_config_from_config_fn_and_checkpoint(
    config_fn=config_fn, checkpoint_fn=checkpoint_fn
)

# load datasets config
datasets_config = convert_datasets_config_to_inference(
    config["datasets"],
    ds_fn=ds_fn,
)

# load dataloader
optim_config = {"device": "cuda", "dtype": torch.float32}
# global_config = {"nthetas": 65, "n_radios": 2, "seed": 0, "beamformer_input": True}

train_dataloader, val_dataloader = load_dataloaders(
    datasets_config, optim_config, config["global"], step=0, epoch=0
)

# run inference
model.eval()
with torch.no_grad():
    for _, val_batch_data in enumerate(tqdm(val_dataloader, leave=False)):
        val_batch_data = val_batch_data.to(config["optim"]["device"])
        output = model(val_batch_data)
        break

In [ ]:
output["single"].shape

In [ ]:
output["single"].shape, output["paired"].shape

In [ ]:
config

In [ ]:
from functools import partial
from spf.dataset.spf_dataset import v5_collate_keys_fast, v5spfdataset
from spf.scripts.train_single_point import global_config_to_keys_used


ds = v5spfdataset(
    ds_fn,
    nthetas=65,
    ignore_qc=True,
    precompute_cache=datasets_config["precompute_cache"],
    empirical_data_fn=datasets_config["empirical_data_fn"],
    paired=True,
    skip_fields=set(["signal_matrix"]),
)

keys_to_get = global_config_to_keys_used(global_config=global_config)
single_example = v5_collate_keys_fast(keys_to_get, [ds[0]]).to(optim_config["device"])
output2 = model(single_example)

In [ ]:
(output["single"] == output2["single"]).all()